<a href="https://colab.research.google.com/github/karunyavalar/TNSDC-_Generative-AI/blob/main/stylGAN2_for_high_fidelity_portrait_generation_and_editing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'rezero-rem-anime-faces-for-gan-training:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1899492%2F3119506%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240329%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240329T115457Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D50a6ace6a65f1d80788403268b0c7c5318f6920c43ffe7063fc46f1910c884e4ba54bd0d5291b897e37425415e858b5e3214606384dc0cdf7b5aae09f44314bede548483326b00c18d6c6f653fdb8457374a6f8f8e3ad994fc574550fa08a92638250fdb038347a46eb88f44a35f114ed2a3d0b150587618483379030b05d584dfe2d9cc79ae70fc62e141c7fa3312fb7d4e58632bd8823d60053b1b0d7889f8c52ad772ca05def48e772d460b0cf24adf6723546f4a9ac54acee1f98e483601e3f6781d469714fc5d7370926fa37277fb6870140c4f6369a7cf56dddff1d7e00ea5267858636ea5e2e437d82653a2d8f722c8097e479b4df5ebf248537473a3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1150368645 bytes downloaded
Downloaded and uncompressed: rezero-rem-anime-faces-for-gan-training
Data source import complete.


## Train your own GAN in a few lines of code!
See original StyleGAN2 ADA github repo here: https://github.com/NVlabs/stylegan2-ada-pytorch

By using a pre-trained model along with data heavy data augmentation, we can train our own GAN with a very limited dataset (<1000 images).

In [ ]:
!pip install pyspng ninja imageio-ffmpeg==0.4.3
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
%cd ./stylegan2-ada-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.4.9
    Uninstalling imageio-ffmpeg-0.4.9:
      Successfully uninstalled imageio-ffmpeg-0.4.9
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 131 (delta 0), reused 1 (delta 0), pack-reused 128
Receiving objects: 100% (131/131), 1.13 MiB | 3.45 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/stylegan2-ada-pytorch


## Download pre-trained model
A pre-trained model on Danbooru portraits will be used

In [ ]:
model_id = "1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez"
network_pkl = './model_%s.pkl' % model_id
gdd.download_file_from_google_drive(file_id=model_id, dest_path=network_pkl)

In [ ]:
!python dataset_tool.py --source=/kaggle/input/rezero-rem-anime-faces-for-gan-training/rem_images/rem_preprocessed_512 --dest=./datasets/rem.zip

100% 725/725 [00:18<00:00, 39.68it/s]


### Training will take a long time, make sure you have enough GPU hours available (minimum 10 hours required for good results)
Do !python train.py --help to see the different optional arguments available.
If resuming training, edit --remsume to your pkl file

In [ ]:
!python train.py --outdir ./results --snap=4 --cfg=paper512 --data=./datasets/rem.zip --augpipe="bg" --mirror=True --metrics=None --resume=./model_1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez.pkl --augpipe="bg"

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 4,
  "network_snapshot_ticks": 4,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/rem.zip",
    "use_labels": false,
    "max_size": 725,
    "xflip": true,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base":

Note the training was cancelled as I have already previously trained a model (see ../input/rezero-rem-anime-faces-for-gan-training/pre-trained/network-snapshot-000108.pkl)